Article: https://medium.com/@abhilashkrish/getting-started-with-pytorch-key-concepts-for-deep-learning-beginners-f0904f252fb2

In [1]:
import torch

In [13]:
# Create a 2D tensor (matrix)
tensor = torch.tensor([[1, 2], [3, 4]])

# Perform operations
tensor_sum = tensor.sum()

# Create a tensor with automatic differentiation enabled
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# Perform some operations 
y = x ** 2
z = y.sum()

# Backpropagate to compute gradients
z.backward()

# Check gradients
print(x.grad)

tensor([2., 4., 6.])


Dataset and DataLoader

In [14]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [17]:
# Define a transform (to convert images to tensor and normalize)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download and load the MNIST training dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Use DataLoader to create batches and shuffle the data
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# Iterate through the DataLoader
for images, labels in train_loader:
    print(f"Batch of images has shape: {images.shape}")  # Output: Batch size, channels, height, width
    break

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>



100.0%


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Batch of images has shape: torch.Size([64, 1, 28, 28])


Model (Neural Network)

In [18]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(28*28, 128)  # Fully connected layer (input: 784, output: 128)
        self.fc2 = nn.Linear(128, 64)     # Fully connected layer (input: 128, output: 64)
        self.fc3 = nn.Linear(64, 10)      # Fully connected layer (input: 64, output: 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = torch.relu(self.fc1(x))  # Apply ReLU after first layer
        x = torch.relu(self.fc2(x))  # Apply ReLU after second layer
        x = self.fc3(x)  # Output layer (no activation function, handled by loss function)
        return x

# Instantiate the model
model = NeuralNet()

Loss Function

In [22]:
import torch.nn as nn

# Define the loss function (cross-entropy)
criterion = nn.CrossEntropyLoss()

# Example of loss computation
outputs = torch.randn(10, 10)  # Random output (10 samples, 10 classes)
labels = torch.randint(0, 10, (10,))  # Random true labels
loss = criterion(outputs, labels)
print(loss.item())

2.9319491386413574


Optimizer

In [23]:
import torch.optim as optim

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training step
for epoch in range(5):  # 5 epochs
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

KeyboardInterrupt: 

Model for Classifiying the MNIST Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Step 1: Load the Dataset (MNIST)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Step 2: Define the Neural Network Model
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # Fully connected layer (28*28 input, 128 neurons)
        self.fc2 = nn.Linear(128, 64)     # Fully connected layer (128 input, 64 neurons)
        self.fc3 = nn.Linear(64, 10)      # Output layer (64 input, 10 output for classes 0-9)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)        # Output, no activation (handled by CrossEntropyLoss)
        return x

# Step 3: Initialize Model, Loss Function, and Optimizer
model = NeuralNet()
criterion = nn.CrossEntropyLoss()  # Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Step 4: Train the Model
def train(num_epochs):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Step 5: Evaluate the Model
def test():
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No need to track gradients during evaluation
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # Get class with highest score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

# Step 6: Run Training and Testing
num_epochs = 5
train(num_epochs)
test()